In [1]:
import sys
sys.path.append('/source/main')

In [2]:
import os
import logging
from datetime import datetime
import time
from itertools import chain

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from naruto_skills.new_voc import Voc
from torch.utils.data import DataLoader, Subset, Dataset
import matplotlib.pyplot as plt

from model_def.siamese_core import SiameseModelCore
from model_def.wrap_core_model import WrapSiameseModelCore
from data_for_train.pool import PoolDocs
from utils import pytorch_utils
from preprocess import preprocessor
from data_for_train.index_dataset import IndexDataset
from data_for_train.positive_dataset import PositiveDataset
from naruto_skills.training_checker import TrainingChecker

In [ ]:
logging.basicConfig(level=logging.INFO)
pd.set_option('display.max_colwidth', -1)

In [ ]:
def docs2input_tensors(docs, device):
    preprocessed_docs = [preprocessor.infer_preprocess(doc) for doc in docs]
#     max_len = max([len(item.split()) for item in preprocessed_docs])
    max_len = 100
    preprocessed_docs = [' '.join(doc.split()[:max_len]) for doc in preprocessed_docs]
    word_input = voc.docs2idx(preprocessed_docs, equal_length=max_len)
    inputs = np.array(word_input)
    input_tensors = torch.from_numpy(inputs)
    input_tensors = input_tensors.to(device)
    return input_tensors

def predict_batch(docs):
    with torch.no_grad():
        input_tensors = docs2input_tensors(docs, device)
        predict_tensor = model(input_tensors)
        predict_np = predict_tensor.cpu().numpy()
        return predict_np[:, 1]

def predict_docs(docs, batch_size):
    return list(chain(*[predict_batch(docs[i: i+batch_size]) for i in tqdm(range(0, len(docs), batch_size))]))

# 1. Data loading

## 1.1 Pool

In [ ]:
def collate_fn(list_data):
    """
    shape == (batch_size, col1, col2, ...)
    """
    data = zip(*list_data)
    data = [np.stack(col, axis=0) for col in data]
    data = [torch.from_numpy(col) for col in data]
    return data
voc = Voc.load('/source/main/vocab/output/voc.pkl')
MAX_LENGTH = 100
BATCH_SIZE = 256
EXP_ID = '25a'

In [ ]:
# df_neg = pd.read_csv('/source/main/data_for_train/output/huge_pool/wiki.csv', nrows=1e6, usecols=['target'])
# df_neg.rename(columns={'target': 'mention'}, inplace=True)
# df_neg.dropna(inplace=True, subset=['mention'])
# df_neg.drop_duplicates(inplace=True, subset=['mention'])
# df_neg = df_neg.iloc[:794323, ]

In [ ]:
# print(df_neg.shape)
# neg = IndexDataset(voc, list(df_neg['mention']), equal_length=MAX_LENGTH)
# neg = PoolDocs(neg)

In [ ]:
df_pool = pd.read_csv('/source/main/data_for_train/output/train/pool.csv', nrows=1e6)
df_pool.dropna(inplace=True, subset=['mention'])
df_pool.drop_duplicates(inplace=True, subset=['mention'])
df_pool = df_pool.iloc[:500000, :]

In [ ]:
print(df_pool.shape)
pool = IndexDataset(voc, list(df_pool['mention']), equal_length=MAX_LENGTH)
pool = PoolDocs(pool)

In [ ]:
len(pool)

## 1.2 Positive data

In [ ]:
df_pos = pd.read_csv('/source/main/data_for_train/output/train/positive_class_1.csv')
df_pos.dropna(inplace=True, subset=['mention'])
df_pos.drop_duplicates(inplace=True, subset=['mention'])

print(df_pos.shape)
pos = IndexDataset(voc, list(df_pos['mention']), equal_length=MAX_LENGTH)
pos = PositiveDataset(pos)

In [ ]:
data_loader = DataLoader(dataset=pos+pool, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [ ]:
data = next(iter(data_loader))

In [ ]:
voc.idx2docs(data[0].cpu().numpy()[:3, :])

In [ ]:
core_model = SiameseModelCore(voc.get_embedding_weights())
model = WrapSiameseModelCore(core_model)

In [ ]:
model.train()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
pytorch_utils.count_parameters(model)

In [ ]:
def train_step(inputs):
    model.train()
    step_loss = model.train_batch(inputs[0], inputs[1])
    return step_loss

In [ ]:
len(data_loader)

In [ ]:
# model.eval()
# # import pdb; pdb.set_trace()
# predict_docs(['giá bao tiền', 'ee', 'Giảm giá sốc'], batch_size=2)

# 2. Training

## 2.1 Epoch 1

In [ ]:
# model.build_stuff_for_training(device)
for epoch_idx in range(1):
    start = time.time()
    for idx, inputs in tqdm(enumerate(data_loader)):
        inputs = [i.to(device) for i in inputs]
        l = train_step(inputs)
        if idx % 10 == 0:
            logging.info('\t Step: %s Loss: %.5f Pos/Neg: %s/%s', idx, l, inputs[1].sum().cpu().item(), 
                         inputs[1].size(0))
    duration = time.time() - start
    logging.info('Epoch %s took %.2f s', epoch_idx, duration)

In [ ]:
# for epoch_idx in range(10):
#     start = time.time()
#     model.train()
#     for idx, inputs in tqdm(enumerate(data_loader)):    
#         inputs = [i.to(device) for i in inputs]
#         l = train_step(inputs)
#     duration = time.time() - start
#     logging.info('Epoch %s took %.2f s', epoch_idx, duration)
    
#     model.eval()    
#     df_pos['pred'] = predict_docs(df_pos['mention'], batch_size=256)
#     df_pool['pred'] = predict_docs(df_pool['mention'], batch_size=256)
    
#     logging.info('Recall: %s/%s=%.4f', (df_pos['pred']>=0.5).sum(), df_pos.shape[0], 
#                  (df_pos['pred']>=0.5).sum()/df_pos.shape[0])
#     logging.info('Ratio on pool: %s/%s=%.4f', (df_pool['pred']>=0.5).sum(), df_pool.shape[0], 
#                  (df_pool['pred']>=0.5).sum()/df_pool.shape[0])
    

In [ ]:
# fig = plt.figure(figsize=(10, 5))

# ax = fig.add_subplot(1, 2, 1)
# df_pos.loc[:500, 'pred'].hist(bins=100, ax=ax)
# ax.set_title('Spy')
# ax.set_xlim(0, 0.9)
# ax.set_ylim(0, 100)

# ax = fig.add_subplot(1, 2, 2)
# df_pos.loc[500:, 'pred'].hist(bins=100, ax=ax)
# ax.set_title('Positive')
# ax.set_xlim(0, 0.9)
# ax.set_ylim(0, 100)

# plt.show()


In [ ]:
# df_pool_social = pd.read_csv('/source/main/data_for_train/output/train/pool.csv')
# df_pool_social.rename(columns={'target': 'mention'}, inplace=True)
# df_pool_social.dropna(inplace=True, subset=['mention'])
# df_pool_social.drop_duplicates(inplace=True, subset=['mention'])
# df_pool_social = df_pool_social.iloc[:794323, ]

In [ ]:
training_checker = TrainingChecker(model, root_dir='/source/main/train/output/saved_models/%s/' % EXP_ID,
                                   init_score=-10000)
training_checker.save_model()

# Analyse

In [ ]:
model.eval()

In [ ]:
df_pos['mention'].sample(3)

In [ ]:
model.eval()
predict_docs(['giá bao tiền', 'ee', 'Giảm giá sốc'], batch_size=1)

In [ ]:
import ast

import pandas as pd
pd.set_option('display.max_colwidth', -1)
from sklearn import metrics

from data_for_train.index_dataset import IndexDataset
from data_for_train.positive_dataset import PositiveDataset
from data_for_train import pool
from naruto_skills.new_voc import Voc

## Recall

### Eval

In [ ]:
df_pos_eval = pd.read_csv('/source/main/data_for_train/output/eval/positive_class_1.csv')
df_pos_eval = df_pos_eval.drop_duplicates(subset=['mention'])

In [ ]:
df_pos_eval['pred'] = predict_docs(list(df_pos_eval['mention']), batch_size=256)
# print(sum(df_pos_eval['pred']>=0.5)/df_pos_eval.shape[0])
# print(df_pos_eval.shape)

In [ ]:
(df_pos_eval['pred']>=0.5).sum()/df_pos_eval.shape[0]

### Test

In [ ]:
df_pos_test = pd.read_csv('/source/main/data_for_train/output/test/positive_class_1.csv')
df_pos_test = df_pos_test.drop_duplicates(subset=['mention'])

In [ ]:
df_pos_test['pred'] = predict_docs(list(df_pos_test['mention']), batch_size=256)
print(sum(df_pos_test['pred']>=0.5)/df_pos_test.shape[0])
print(df_pos_test.shape)

## Score: pr/P(y=1)

### Eval

In [ ]:
df_pool_eval = pd.read_csv('/source/main/data_for_train/output/eval/pool.csv')

In [ ]:
df_pool_eval['pred'] = predict_docs(list(df_pool_eval['mention']), batch_size=256)

# print(sum(df_pool_eval['pred']>=0.5)/df_pool_eval.shape[0])
# print(df_pool_eval.shape)

In [ ]:
sum(df_pool_eval['pred']>=0.5)/df_pool_eval.shape[0]

In [ ]:
df_pool_eval[df_pool_eval['pred']>=0.5].sample(20)

### Test

In [ ]:
df_pool_test = pd.read_csv('/source/main/data_for_train/output/test/pool.csv')

In [ ]:
df_pool_test['pred'] = predict_docs(list(df_pool_test['mention']), batch_size=256)

In [ ]:
print(sum(df_pool_test['pred']>=0.5)/df_pool_test.shape[0])
print(df_pool_test.shape)

In [ ]:
df_pool_eval[df_pool_eval['pred']>=0.5].sample(100).to_csv('%s.csv' % EXP_ID, index=None)